In [27]:
import torch

import model

net = model.Time2Vec(
    input_size=9,
    activation="sin",
    hidden_dim=200,
    out_dim=4,
    batch_size=32,
    lstm_hidden_dim=20,
    lstm_layer=1
)

ckpt = torch.load("best.ckpt")
w = {k[4:]: v for k, v in ckpt["state_dict"].items() if k.startswith("net.")}

net.load_state_dict(w)
net.eval()

Time2Vec(
  (tsv): SineActivation()
  (lstm): LSTM(200, 20, batch_first=True)
  (out_layer): Linear(in_features=20, out_features=4, bias=True)
)

In [28]:

import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("../train_model/data/bitcoin_2017_to_2023.csv")

df = df.dropna()
df = df.reset_index(drop=True)
df["timestamp"] = df["timestamp"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

df = df.drop(columns=["timestamp"])
df = df.reset_index(drop=True)

df_len = len(df)
test_size = 0.2
cut_idx = int(df_len*test_size)
train, val = df.iloc[cut_idx:], df.iloc[:cut_idx]


In [ ]:

scaler = MinMaxScaler(feature_range=(-1, 1))
train = train.copy()
train[train.columns] = scaler.fit_transform(train[train.columns])
val = val.copy()
val[val.columns] = scaler.transform(val[val.columns])


In [9]:
from tqdm import tqdm

window_size = 30
preds = []
labels = []
for index in tqdm(range(len(val) - window_size - 2)):
    x = val.iloc[index: index + window_size].values
    y = val.iloc[index + window_size][["open", "high", "low", "close"]].values

    x = torch.Tensor(x).unsqueeze(dim=0)
    y = torch.Tensor(y).unsqueeze(dim=0)

    pred = net(x)
    
    preds.append(pred.detach().numpy())
    labels.append(y.numpy())

  0%|          | 273/625168 [00:00<15:33, 669.22it/s]

100%|██████████| 625168/625168 [15:53<00:00, 655.47it/s]


In [29]:
import numpy as np

def result_to_df(result):
    result = np.array(result).squeeze()

    return pd.DataFrame(result, columns=["open", "high", "low", "close"])

pred_df = result_to_df(preds)
label_df = result_to_df(labels)

In [30]:
scaler.fit(train[["open", "high", "low", "close"]])

MinMaxScaler(feature_range=(-1, 1))

In [32]:
inv_pred_df = scaler.inverse_transform(pred_df)
inv_label_df = scaler.inverse_transform(label_df)

In [42]:
np.sum(inv_pred_df - inv_label_df) / len(inv_label_df)

97.52813323778568

In [39]:
inv_label_df

array([[28880.99 , 28884.32 , 28879.389, 28879.389],
       [28876.34 , 28881.09 , 28876.33 , 28881.   ],
       [28882.389, 28882.389, 28876.33 , 28876.34 ],
       ...,
       [28984.85 , 28989.95 , 28956.85 , 28982.7  ],
       [28945.49 , 28990.   , 28945.441, 28984.85 ],
       [28927.658, 28969.43 , 28913.049, 28945.5  ]], dtype=float32)

In [40]:
inv_pred_df

array([[28802.936, 28825.955, 28780.098, 28803.11 ],
       [28805.906, 28828.924, 28783.07 , 28806.082],
       [28804.994, 28828.014, 28782.158, 28805.17 ],
       ...,
       [28912.37 , 28935.352, 28889.559, 28912.516],
       [28913.662, 28936.645, 28890.852, 28913.809],
       [28910.355, 28933.334, 28887.543, 28910.502]], dtype=float32)

In [53]:
tsv = net.tsv

In [ ]:
from tqdm import tqdm

window_size = 30
tsv_preds = []
tsv_labels = []
for index in tqdm(range(len(val) - window_size - 2)):
    x = val.iloc[index: index + window_size].values
    y = val.iloc[index + window_size][["open", "high", "low", "close"]].values

    x = torch.Tensor(x).unsqueeze(dim=0)
    y = torch.Tensor(y).unsqueeze(dim=0)

    pred = net(x)
    
    tsv_preds.append(pred.detach().numpy())
    tsv_labels.append(y.numpy())